In [1]:
import numpy as np
import pandas as pd
from template import *
from template2 import *
from time import time
from itertools import combinations

Juan Manuel Jimenez  
Ian Chona  
Camilo Franco Beltran  


#  Sheet -9-

1.Write functions sup(D,X,Y=None), conf(D,X,Y), lift(D,X,Y=None), leverage(D,X,Y),
jaccard(D,X,Y), conviction(D,X,Y), oddsRatio(D,X,Y), imp(D,X,Y) that compute
for any rule X 􀀀! Y the respective metric given the dataset D, where D is a list
of lists of item IDs. Note that, for the support and lift, the conclusion is optional.
This is to allow the calculation of support and lift of patterns (itemsets). Add a
function getRuleMetric(D,X,Y,metric) that computes the metric metric (given as a
string name in fsup; conf ; lift ; leverage; jaccard; conviction; oddsratio; impg) for the rule
X 􀀀! Y in the data D.

In [71]:
def sup(D,X,Y=None):
    if Y==None:
        Y=[]
    db=pd.DataFrame(D).values    
    XY=sorted(np.concatenate((X, Y), axis=0))    
    return np.count_nonzero(np.sum(np.isin(db,XY),axis=1) ==len(XY))

def conf(D,X,Y):    
    supXY=sup(D,X,Y)
    supX=sup(D,X)
    return supXY/supX

def lift(D,X,Y=None):
    if Y==None:
        supX=sup(D,X)
        supXi=1
        lenD=len(D)
        for element in X:
            supXi=supXi*(sup(D,[element])/lenD)
        confidence=(supX/lenD)/supXi
    else:
        
        confXY=conf(D,X,Y)
        rsupY=sup(D,Y)/len(D)
        confidence=confXY/rsupY
    return confidence

def leverage(D,X,Y):
    lenD=len(D)
    rsupXY=sup(D,X,Y)/lenD
    rsupX=sup(D,X)/lenD
    rsupY=sup(D,Y)/lenD
    return rsupXY-rsupX*rsupY

def jaccard(D,X,Y):
    supXY=sup(D,X,Y)
    supX=sup(D,X)
    supY=sup(D,Y)
    return supXY/(supX+supY-supXY)

def conviction(D,X,Y):
    rsupY=sup(D,Y)/len(D)
    confXY=conf(D,X,Y)    
    return (1-rsupY)/(1-confXY)

def oddsRatio(D,X,Y):
    supXY=sup(D,X,Y)
    supX=sup(D,X)
    supY=sup(D,Y)
    supnXY=supY-supXY
    supXnY=supX-supXY
    supnXnY=(len(D)-supX)-(supnXY)
    return (supXY*supnXnY)/(supXnY*supnXY)

def imp(D,X,Y):# [217, 546, 661, 923, 947] probar con esta regla para verificar que funciona
    premises=powerset(X)
    confXY=conf(db,X,Y)
    improve=X+Y
    
    for j in premises:
        if len(j)<len(X) and  j:
            confWY=conf(db,j,conclusion)
            
            if confWY>=confXY:
                improve=j+Y 
                
    return improve
def getRuleMetric(D,X,Y,metric):
    my_metrics = {"sup":sup,"conf":conf,"lift":lift,"leverage":leverage,"jaccard":jaccard,"conviction":conviction,"oddsRatio":oddsRatio,"imp":imp}
    return my_metrics[metric](D,X,Y)


2. Write a function getRuleBasedScore(D,X,metric,agg) that computes for all bi-partitions
of X the score of metric metric (given as a string name in {sup; conf ; lift ; leverage;
jaccard; conviction; oddsratio; imp}) and aggregates it based on one of the three keywords
min, max, avg for the agg parameter.

In [7]:
def getRuleBasedScore(D,X,metric,agg):
    
    a=powerset(X)
    lenA=len(a)
    results=[]
    for i in range(1,lenA-1):
        rule=[a[i],a[lenA-i-1]]        
        results.append(getRuleMetric(D,a[i],a[lenA-i-1],metric))
    
    measures={"max":max(results),"min":min(results),"avg":sum(results)/len(results)}
    
    return measures[agg]

3. Write a function filterProductiveRules(D, R) that takes a database D as above and
a list R of tuples (X,Y ) corresponding to rules X -> Y . It should return all rules of R
that are productive.

In [8]:
db=readTransactionalDatabase("shop.dat")
def filterProductiveRules(D,R):
    productiveRules=[]
    for rule in R:
        minLift=getRuleBasedScore(D,rule,"lift","min")
        minLeverage=getRuleBasedScore(D,rule,"leverage","min")
        if minLift>1 and minLeverage>0:
            productiveRules.append(rule)
            
    return productiveRules
filterProductiveRules(db,[[2,3,4,5],[1,2,3]])

[]

4. Use the algorithms in the template file to compute all the strong rules of the shop
database and filter them so that you only retain productive ones; use minsup = 500 and
minconf = 0.95. Now create for each 3-combination of rule metrics we have seen a 3D
scatter plot that contains one (labeled) point for each rule. Based on these plots, can
you identify particularly useful rules? Which are they and why?

In [67]:
db=readTransactionalDatabase("shop.dat")
rules=eclat(db, 1000)
print(rules)
rulesToImprove=[row[0] for row in rules if len(row[0])>2]
print(rulesToImprove,"to improve")

[([1], 1535), ([4], 1394), ([5], 1094), ([6], 2149), ([8], 3090), ([10], 1351), ([12], 3415), ([17], 1683), ([21], 2666), ([25], 1395), ([27], 2165), ([28], 1454), ([31], 1666), ([32], 4248), ([33], 1460), ([35], 1984), ([37], 1249), ([38], 2402), ([39], 4258), ([39, 704], 1107), ([39, 704, 825], 1035), ([39, 825], 1187), ([41], 1353), ([43], 1721), ([45], 1728), ([48], 2472), ([51], 1612), ([52], 1983), ([54], 2595), ([55], 1959), ([57], 2743), ([58], 1330), ([68], 1601), ([69], 2370), ([70], 2411), ([71], 3507), ([72], 2852), ([73], 2179), ([75], 3151), ([78], 2471), ([85], 1555), ([90], 1875), ([93], 2777), ([94], 1201), ([97], 1466), ([100], 1749), ([104], 1158), ([105], 1100), ([110], 1801), ([111], 1171), ([112], 2680), ([115], 1775), ([116], 2193), ([120], 4973), ([122], 1081), ([125], 1287), ([126], 1075), ([129], 1547), ([130], 1711), ([132], 2641), ([140], 2687), ([143], 1417), ([145], 4559), ([147], 1383), ([151], 2611), ([154], 1447), ([157], 1140), ([161], 2320), ([162], 1

In [72]:
for i in rulesToImprove:
    premise=i.copy()
    premise.pop()
    conclusion=[i[-1]]
    i=imp(db,premise,conclusion)
    print()
print(rulesToImprove)
#[1,2,3,4,5]
#[1,2,3]-[4,5]
#[1,2]
#[1,3]


[[39, 704, 825]]


In [73]:
imp(db,[39 ,  704] ,[ 825 ])

[39, 704, 825]